In [1]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py PyMuPDF sentence_transformers python-dotenv langchain_intro


ERROR: Could not find a version that satisfies the requirement langchain_intro (from versions: none)
ERROR: No matching distribution found for langchain_intro


In [2]:
## index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.docstore.document import Document

# Load PDF documents
pdf_file_paths = ["./data/Medical Policy FY 23-24.pdf", "./data/Provident fund policy.pdf"]

docs_list = []
for pdf_path in pdf_file_paths:
    loader = PyMuPDFLoader(pdf_path)
    try:
        loaded_docs = loader.load()
        docs_list.extend(loaded_docs)
    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)
doc_splitter = text_splitter.split_documents(docs_list)

# Filter and clean metadata
filtered_doc = []
for doc in doc_splitter:
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_doc.append(Document(page_content=doc.page_content, metadata=clean_metadata))

# Add to vectorDB
embedding = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf", gpt4all_kwargs={'allow_download': 'True'})
vectorstore = Chroma.from_documents(
    documents=filtered_doc,
    collection_name="rag-chroma",
    embedding=embedding,
)

retriever = vectorstore.as_retriever()

In [3]:
# local_llm = 'gemma'
local_llm = 'llama3'
# local_llm = 'mistral'

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model=local_llm, temperature=0)

output_parser = StrOutputParser()


In [5]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain.schema.runnable import RunnablePassthrough

context_template_str = """system You are an assistant for question-answering tasks. Use the following context to answer the question. Avoid phrases like "Based on the provided context". Explain the answer in the end. and make a heading with paragraph.
Question: {question}
Context: {context}
Answer: assistant"""

context_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context", "question"],
        template=context_template_str,
    )
)

context_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="{question}",
    )
)

context_messages = [context_system_prompt, context_human_prompt]

context_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=context_messages,
)

context_chain = {"context": retriever,
                 "question": RunnablePassthrough()} | context_prompt_template | llm | output_parser

context_question = "Does original receipts must be submitted?"
context_answer = context_chain.invoke(context_question)
print(context_answer)

**Original Receipt Submission Requirement**

According to the Provident Fund Policy V2.0, original receipts must be submitted within 30 days of loan disbursement for construction on a plot. This is stated in point 2 under "Documents to be submitted" on page 6.

However, it's worth noting that this requirement only applies to construction on a plot and not to other types of withdrawals or expenses. For medicines, tests, procedures, etc., original receipts are required as per the Medical Policy FY 23-24.

**Key Takeaway**

Original receipts must be submitted within 30 days of loan disbursement for construction on a plot, but this requirement may vary depending on the type of expense or withdrawal.


In [6]:
no_context_system_template_str = """system You are an assistant for question-answering tasks. Answer the question concisely and directly. If you dont know the answer the say dont know.
Question: {question}
Answer: assistant"""

no_context_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template=no_context_system_template_str,
    )
)

no_context_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="{question}",
    )
)

no_context_messages = [no_context_system_prompt, no_context_human_prompt]

no_context_prompt_template = ChatPromptTemplate(
    input_variables=["question"],
    messages=no_context_messages,
)

no_context_chain = {"question": RunnablePassthrough()} | no_context_prompt_template | llm | output_parser

no_context_question = "what is 911 event"
no_context_answer = no_context_chain.invoke(no_context_question)
print(no_context_answer)

The 9/11 event, also known as the September 11 attacks, was a series of coordinated terrorist attacks that occurred on September 11, 2001. On that day, four commercial airplanes were hijacked by terrorists affiliated with al-Qaeda. Two planes crashed into the Twin Towers of the World Trade Center in New York City, causing massive damage and loss of life. Another plane crashed into the Pentagon in Arlington, Virginia, and the fourth plane, believed to be heading for the White House or the U.S. Capitol Building, crashed in a field in Pennsylvania after passengers attempted to overpower the hijackers. In total, nearly 3,000 people were killed in the attacks.


In [7]:
import random
import time


def get_current_wait_time(hospital: str) -> int | str:
    """Dummy function to generate fake wait times"""

    if hospital not in ["A", "B", "C", "D"]:
        return f"Hospital {hospital} does not exist"

    # Simulate API call delay
    time.sleep(1)

    return random.randint(0, 10000)

In [18]:
from langchain.agents import (
    create_openai_functions_agent,
    Tool,
    AgentExecutor,
)
from langchain import hub

tools = [
    Tool(
        name="MedicalPolicy",
        func=no_context_chain.invoke,
        description="""Useful when you need to answer questions about the Medical Policy FY 23-24.
        This includes eligibility, coverage details, claims process, and other related information.
        Pass the entire question as input to the tool. For instance, if the question is "Does original receipts must be submitted?",
        the input should be "Does original receipts must be submitted?".
        """,
    ),
    Tool(
        name="ProvidentFundPolicy",
        func=no_context_chain.invoke,
        description="""Useful when you need to answer questions about the Provident Fund Policy.
        This includes eligibility, contributions, profit on contributions, withdrawals, and loans.
        Pass the entire question as input to the tool. For instance, if the question is "What is the eligibility for the provident fund?",
        the input should be "What is the eligibility for the provident fund?".
        """,
    ),
    Tool(
        name="Waits",
        func=get_current_wait_time,
        description="""Use when asked about current wait times
        at a specific hospital. This tool can only get the current
        wait time at a hospital and does not have any information about
        aggregate or historical wait times. This tool returns wait times in
        minutes. Do not pass the word "hospital" as input,
        only the hospital name itself. For instance, if the question is
        "What is the wait time at hospital A?", the input should be "A".
        """,
    ),
]


In [22]:
agent_prompt = hub.pull("hwchase17/openai-functions-agent")

hospital_agent = create_openai_functions_agent(
    llm=llm,
    prompt=agent_prompt,
    tools=tools,
)

hospital_agent_executor = AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

hospital_agent_executor.invoke(
    {"input": "What is the current wait time at hospital A?"}
)



> Entering new AgentExecutor chain...
I'm happy to help! However, I'm just an AI and don't have real-time access to specific hospitals' wait times. But I can suggest some ways for you to find out the current wait time at Hospital A:

1. Check their website: You can visit Hospital A's official website and look for a "Wait Times" or "Patient Information" section, which may provide an estimate of the current wait time.
2. Call the hospital directly: Reach out to Hospital A's main phone number or patient information line and ask about the current wait time. They should be able to provide you with an update.
3. Use a hospital finder app: There are several apps available that allow you to search for hospitals, check wait times, and even book appointments. Some popular options include Zocdoc, Healthgrades, and RateMDs.

Remember to always verify the accuracy of any information you find online or through phone calls. If you have any further questions or concerns, feel free to ask!

> Finishe

{'input': 'What is the current wait time at hospital A?',
 'output': 'I\'m happy to help! However, I\'m just an AI and don\'t have real-time access to specific hospitals\' wait times. But I can suggest some ways for you to find out the current wait time at Hospital A:\n\n1. Check their website: You can visit Hospital A\'s official website and look for a "Wait Times" or "Patient Information" section, which may provide an estimate of the current wait time.\n2. Call the hospital directly: Reach out to Hospital A\'s main phone number or patient information line and ask about the current wait time. They should be able to provide you with an update.\n3. Use a hospital finder app: There are several apps available that allow you to search for hospitals, check wait times, and even book appointments. Some popular options include Zocdoc, Healthgrades, and RateMDs.\n\nRemember to always verify the accuracy of any information you find online or through phone calls. If you have any further questions